<a href="https://colab.research.google.com/github/2018007956/HYU/blob/main/Deep_Learning/11_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. RNN을 이용한 character prediction

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [2]:
sentence = ' seoul is the capital of south korea'
char_set = list(set(sentence))
char_dic = {c:i for i,c in enumerate(char_set)}

In [4]:
char_set

['u', 'a', 'p', 'i', 'o', 'h', 'f', 'k', 'e', ' ', 's', 'l', 't', 'r', 'c']

In [3]:
char_dic

{'u': 0,
 'a': 1,
 'p': 2,
 'i': 3,
 'o': 4,
 'h': 5,
 'f': 6,
 'k': 7,
 'e': 8,
 ' ': 9,
 's': 10,
 'l': 11,
 't': 12,
 'r': 13,
 'c': 14}

In [5]:
vocab_sz = len(char_dic)
hidden_sz = len(char_dic)
input_sz = len(char_dic)

In [9]:
'''
Sentence의 character index로 변경
input:" seoul is the capital of south kore"의 index input을 one hot vector로 변경
target:"seoul is the capital of south korea:
'''
sen_idx = [char_dic[c] for c in sentence]
x_idx = sen_idx[:-1]
x_one_hot = [[np.eye(vocab_sz)[x] for x in x_idx]]
y_data = [sen_idx[1:]]

In [10]:
print(sen_idx)
print(x_idx)
print(x_one_hot)
print(y_data)

[9, 10, 8, 4, 0, 11, 9, 3, 10, 9, 12, 5, 8, 9, 14, 1, 2, 3, 12, 1, 11, 9, 4, 6, 9, 10, 4, 0, 12, 5, 9, 7, 4, 13, 8, 1]
[9, 10, 8, 4, 0, 11, 9, 3, 10, 9, 12, 5, 8, 9, 14, 1, 2, 3, 12, 1, 11, 9, 4, 6, 9, 10, 4, 0, 12, 5, 9, 7, 4, 13, 8]
[[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]), array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]), arra

In [13]:
x_train = torch.FloatTensor(x_one_hot)
y_train = torch.LongTensor(y_data)

<ipython-input-13-9278ed7c0bad>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  x_train = torch.FloatTensor(x_one_hot)


In [16]:
class Rnn(nn.Module):
  def __init__(self, input_size, hidden_size, vocab_size):
    super(Rnn, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.vocab_size = vocab_size

    self.rnn = nn.RNN(input_size=self.input_size, hidden_size=self.hidden_size, batch_first=True)
    self.linear = nn.Linear(self.hidden_size, self.vocab_size)

  def forward(self, x):
    outputs, _ = self.rnn(x)
    x = self.linear(outputs)

    return x

model = Rnn(input_size = input_sz, hidden_size=hidden_sz, vocab_size=vocab_sz).to(device)

In [15]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [19]:
epochs = 500

for epoch in range(epochs):
  model.train()

  outputs = model(x_train.to(device)) # forward propagation
  loss = criterion(outputs.view(-1, hidden_sz), y_train.view(-1).to(device))

  optimizer.zero_grad()
  loss.backward() # backward propagation
  optimizer.step() # update parameters

  result = outputs.data.cpu().numpy().argmax(axis=2)
  result_str = ''.join([char_set[idx] for idx in np.squeeze(result)])

  if epoch % 50 == 0 or epoch == epochs-1:
    print('loss : {} prediction : {}'.format(loss, result_str))

loss : 2.747002363204956 prediction : cppppaauucpapauppuppuafpcppppapupap
loss : 2.747002363204956 prediction : cppppaauucpapauppuppuafpcppppapupap
loss : 2.747002363204956 prediction : cppppaauucpapauppuppuafpcppppapupap
loss : 2.747002363204956 prediction : cppppaauucpapauppuppuafpcppppapupap
loss : 2.747002363204956 prediction : cppppaauucpapauppuppuafpcppppapupap
loss : 2.747002363204956 prediction : cppppaauucpapauppuppuafpcppppapupap
loss : 2.747002363204956 prediction : cppppaauucpapauppuppuafpcppppapupap
loss : 2.747002363204956 prediction : cppppaauucpapauppuppuafpcppppapupap
loss : 2.747002363204956 prediction : cppppaauucpapauppuppuafpcppppapupap
loss : 2.747002363204956 prediction : cppppaauucpapauppuppuafpcppppapupap
loss : 2.747002363204956 prediction : cppppaauucpapauppuppuafpcppppapupap


## 2. LSTM